In [2]:
library(shiny)
library(dplyr)
library(ggplot2)
library(caret)
library(shinythemes)
library(shinyjs)

# Load dataset with error handling
load_data <- function(file_path) {
  if (!file.exists(file_path)) {
    stop("File does not exist: ", file_path)
  }
  df <- read.csv(file_path, stringsAsFactors = FALSE)
  colnames(df) <- make.names(colnames(df))
  colnames(df) <- gsub("[.()]", "_", colnames(df))  # Replace dots and parentheses with underscores
  return(df)
}

# Data Preprocessing
preprocess_data <- function(df) {
  df$Industry <- as.factor(df$Industry)
  df <- df %>%
    select(Startup_Name, Valuation__USD_, Growth_Rate____, Industry, Investment_Amount__USD_, Number_of_Investors)
  return(df)
}

# Train Regression Model
train_model <- function(df) {
  set.seed(123)
  train_index <- createDataPartition(df$Valuation__USD_, p = 0.8, list = FALSE)
  train_data <- df[train_index, ]
  model <- lm(Growth_Rate____ ~ Valuation__USD_ + Industry + Investment_Amount__USD_ + Number_of_Investors, data = train_data)
  return(model)
}

# Suggest Startups Based on Valuation and Growth Rate
suggest_startups <- function(df, valuation_threshold, growth_rate_threshold, industry_type) {
  suggested_startups <- df %>%
    filter(Valuation__USD_ >= valuation_threshold & Growth_Rate____ >= growth_rate_threshold & Industry == industry_type) %>%
    arrange(desc(Growth_Rate____)) %>%
    head(10)
  return(suggested_startups)
}


In [ ]:
# UI Definition
ui <- fluidPage(
  titlePanel("Startup Growth Investment Suggestions"),
  useShinyjs(),
  theme = shinytheme("darkly"),
  sidebarLayout(
    sidebarPanel(
      selectInput("industry_type", "Select Industry Type:", 
                  choices = c("AI", "Biotech", "Blockchain", "E-commerce", "EdTech", "Fintech", "HealthTech", "SaaS"), 
                  selected = "AI"),
      numericInput("valuation_threshold", "Valuation Threshold (USD):", value = 100000000),
      numericInput("growth_rate_threshold", "Growth Rate Threshold (%):", value = 80),
      actionButton("submit", "Get Suggestions")
    ),
    
    mainPanel(
      tableOutput("suggested_startups"),
      br(),
      textOutput("further_steps"),
      plotOutput("industry_plot")  # Add plot output for industry activity
    )
  )
)

# Server Logic
server <- function(input, output, session) {
  file_path <- "C:/Users/DJ COMPUTERS/OneDrive/Desktop/LaunchX/startup_growth_investment_data.csv"  # Change this to your file path
  df <- load_data(file_path)
  df <- preprocess_data(df)
  model <- train_model(df)
  
  observeEvent(input$submit, {
    suggested_startups <- suggest_startups(df, input$valuation_threshold, input$growth_rate_threshold, input$industry_type)
    output$suggested_startups <- renderTable({
      suggested_startups
    })
    
    output$further_steps <- renderText({
      "You can check on these startups for potential investment opportunities."
    })
    
    # Render the industry activity plot
    output$industry_plot <- renderPlot({
      industry_counts <- df %>%
        group_by(Industry) %>%
        summarise(Count = n())
      
      ggplot(industry_counts, aes(x = reorder(Industry, -Count), y = Count)) +
        geom_bar(stat = "identity", fill = "steelblue") +
        labs(title = "Number of Startups by Industry", x = "Industry", y = "Number of Startups") +
        theme_minimal() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1))
    })
  })
}

# Run the application 
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:4471

